In [5]:
#!/usr/bin/env python
#
# Simple DMRG for square ice
#  - Infinite system algorithm
#  - Finite system algorithm
#
# This code is based on the DMRG Tutorial of:
# Copyright 2013 James R. Garrison and Ryan V. Mishmash.
# Open source under the MIT license.  Source code at
# <https://github.com/simple-dmrg/simple-dmrg/>

# This code will run under any version of Python >= 2.6.  The following line
# provides consistency between python2 and python3.
from __future__ import print_function, division  # requires Python >= 2.6

# numpy and scipy imports
import numpy as np
from scipy.sparse import kron, identity
from scipy.sparse.linalg import eigsh  # Lanczos routine from ARPACK
import matplotlib.pyplot as plt
# We will use python's "namedtuple" to represent the Block and EnlargedBlock
# objects
from collections import namedtuple

# Squared Ice - MPS

This notebook implements the Squared Ice Model in a DMRG procedure

## Hamiltonian and grid definition
We define the Square Ice Hamiltonian

\begin{equation}
    H = \sum_\square (-f_\square + \lambda f^2_\square)\,,\label{eq:hamiltonian}
\end{equation}
where we some over all plaquettes. The plaquette operator is defined as:
\begin{equation}\label{eq:plaquette}
       f_\square = \sigma^+_{\mu_1}\sigma^+_{\mu_2}\sigma^-_{\mu_3}\sigma^-_{\mu_4}\, + h.c..
\end{equation}
The Hamiltonian eq.\eqref{eq:hamiltonian} is invariant under the local symmetry:
\begin{equation}\label{eq:}
    G_\nu = \sum_{\hat{i}\in\{\hat{x},\hat{y}\}} ( \sigma_{\nu-\hat{i}/2}-\sigma_{\nu+\hat{i}/2})
\end{equation}
which counts the difference between in and outgoing arrows at vertex $\nu$.
In this work 
\begin{equation}\label{eq:ice_rule}
G_\nu=0 \qquad\text{ for all }\qquad \nu \in \Omega
\end{equation}

For the cylindrical lattice:
\begin{align}\label{eq:geometry}
\Omega=\{\nu = (n,m) |\qquad n\in\{1,\dots,L_x\}, \;  m \in\{1,\dots L_y\}\}
\end{align}


In [6]:
## params not used yet
class params_type:
    """ 
    Class of all parameters needed for the Simulation
    """
   
    class geometry:
        Lx=4 # integer number of sites in x direction
        Ly=5 # integer number of sites in y direction
        
    def __init__(self, Lx=5, Ly=10):
        self.geometry.Lx=Lx
        self.geometry.Ly=Ly

Some helping functions are needed:

In [7]:
Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])

def is_valid_block(block):
    for op in block.operator_dict.values():
        # here we have to differentiate between an operator list and a
        # single operator
        if type(op) is list:
            for op_element in op:
                if op_element.shape[0] != block.basis_size or op_element.shape[1] != block.basis_size:
                    return False
        else:
            if op.shape[0] != block.basis_size or op.shape[1] != block.basis_size:
                return False
    return True

def mkron( *kron_list):
    A = kron_list[0]
    for i,val in  enumerate(kron_list[1:],start=1):
        A = np.kron(A, val)  
    
    return A

# This function should test the same exact things, so there is no need to
# repeat its definition.
is_valid_enlarged_block = is_valid_block

## Hilbert-Space
In the absence of the ice rule eq.\eqref{eq:ice_rule} the hilbertspace becomes $2^{2\cdot L_xL_y}$ dimensional  and the linear combination of every state is given by:
\begin{equation}\label{eq:hilbertspace}
|\psi\rangle = \sum_{i_1,i_2,\dots,i_{L_x}}A_{i_1,i_2,\dots,i_{L_x}} \,|i_1\rangle|i_2\rangle\dots|i_{L_x}\rangle
\end{equation}
where $i_n=1,2,\dots,2^{L_y}$ labels the corresponding quantum state.
First we construct all possible states $|i_n\rangle\in\{|l_n\rangle|v_n\rangle|r_n\rangle\}$ which fullfill the Gauss-law \eqref{eq:ice_rule}.

## Local Hamiltonian
For the MPS we have to rewrite the Hamiltonian of the system in the nearest neighbour setting. The local Hamiltonian $H_{n,n+1}$ thus defines the interaction between the states at site $|i_n\rangle$ and $|i_{n+1}\rangle$. The Hamilton operator \eqref{eq:hamiltonian} consists of 4 terms. Where on each site we have $m=1,\dots,L_y$ possible interactions. Thus the hamiltonian consists of $4L_y$ Kronecker products:
\begin{align}
H_{n,n+1}= \sum_{j=1}^4\sum_{m=1}^{L_y}h^{(j)}_{\sqsubset,n,m} \otimes h^{(j)}_{\sqsupset,n+1,m}\label{eq:Hloc}
\end{align}
In the code we simply loop over the terms on the left ($n$) and right ($n+1$) site:

In [8]:
def H2(hleft_list, hright_list):  # two-site part of H
    """Given the operators S^z and S^+ on two sites in different Hilbert spaces
    (e.g. two blocks), returns a Kronecker product representing the
    corresponding two-site term in the Hamiltonian that joins the two sites.
    """
    Hinteract = 0
    for hleft,hright in zip(hleft_list,hright_list):    
        Hinteract += kron(hleft,hright) 
    
    J = Jz = 1.
    Hinteract = J * Hinteract
    return Hinteract


## Plaquette-operators
To identify the different local interaction terms in the Hamilton operator  \eqref{eq:Hloc} with \eqref{eq:hamiltonian} we rewrite the 
plaquette-operator into our computational basis $|i_n\rangle$. A plaquette operator defines our nearest neighbor interaction between state 
$|i_n\rangle$ and $|i_{n+1}\rangle$

\begin{align}
f_\square &= f_{\sqsubset,n,m}\otimes f_{\sqsupset,n,m} + h.c.\\
f_{\sqsubset,n,m} &= \sigma^-_{r,n,m+1}\sigma^-_{v,n,m+1}\sigma^+_{r,n,m}\\
f_{\sqsupset,n+1,m} &= \sigma^+_{l,n+1,m+1}\sigma^+_{v,n+1,m+1}\sigma^-_{l,n+1,m}
\end{align}

Comparing this to eq.\colon\eqref{eq:Hloc} yields:

\begin{align}
h^{(1)}_{\sqsubset,n,m} &=- f_{\sqsubset,n,m}\quad
&h^{(1)}_{\sqsupset,n+1,m} = f_{\sqsupset,n+1,m}\\
%%%%
h^{(2)}_{\sqsubset,n,m} &=- f^\dagger_{\sqsubset,n,m}
\quad
&h^{(2)}_{\sqsupset,n+1,m} = f^\dagger_{\sqsupset,n+1,m}\\
%%%%
h^{(3)}_{\sqsubset,n,m} &=\lambda f^\dagger_{\sqsubset,n,m}f_{\sqsubset,n,m}
\quad
&h^{(3)}_{\sqsupset,n+1,m} = f^\dagger_{\sqsupset,n+1,m}f_{\sqsupset,n+1,m}\\
%%%%
h^{(4)}_{\sqsubset,n,m} &=
\lambda f_{\sqsubset,n,m}f^\dagger_{\sqsubset,n,m}
\quad
&h^{(4)}_{\sqsupset,n+1,m} = 
f_{\sqsupset,n+1,m}f^\dagger_{\sqsupset,n+1,m}\\
\end{align}

For example in our $L_y=2$ system we get $64 \times 64$ size Operators :

\begin{align}
h^{(1)}_{\sqsubset,n,m} &=-\sigma^+\otimes\sigma^-\otimes\sigma^+\otimes I_2 \otimes I_2 \otimes I_2 \in \mathbb{R}^{2^6,2^6}\\
h^{(1)}_{\sqsupset,n,m} &=I_2 \otimes I_2 \otimes \sigma^+ \otimes I_2\otimes\sigma^-\otimes\sigma^+ \in \mathbb{R}^{2^6,2^6}\\
\end{align}
Here:
\begin{align}
\sigma^+=
\begin{pmatrix}
0 & 1 \\
0 & 0
\end{pmatrix}
\quad
\sigma^-=
\begin{pmatrix}
0 & 0 \\
1 & 0
\end{pmatrix}
\qquad
I_2 = 
\begin{pmatrix}
1 & 0 \\
0 & 1
\end{pmatrix}
\end{align}


In [9]:
# Model-specific code for 2 x L chain
model_d = 2**6  # single-site basis size

I2 = np.eye(2)

Sp1 = np.array([[0, 1], [0, 0]], dtype='d')  # single-site S^+
Sm1 = Sp1.conjugate().transpose()
Sz1 = np.array([[0.5, 0], [0, -0.5]], dtype='d')  # single-site S^z

Plq_right = [ mkron(I2,I2,Sp1,I2,Sm1,Sp1), mkron(I2,I2,I2,Sp1,Sp1,Sm1)] #right half plaquette
Plq_right_deg = [mkron(I2,I2,Sm1,I2,Sp1,Sm1),mkron(I2,I2,I2,Sm1,Sm1,Sp1)] #right half plaquette

Plq_left = [-mkron(Sp1,Sm1,Sp1,I2,I2,I2),-mkron(Sm1,Sp1,I2,Sp1,I2,I2)]
Plq_left_deg = [-mkron(Sm1,Sp1,Sm1,I2,I2,I2), -mkron(Sp1,Sm1,I2,Sm1,I2,I2)]

H1 = np.zeros([2**6, 2**6] , dtype='d')  # single-site portion of H is zero


In the DMRG we build the tensor in the following way:

\begin{align}
H = 
\end{align}

In [ ]:

# conn refers to the connection operator, that is, the operator on the edge of
# the block, on the interior of the chain.  We need to be able to represent S^z
# and S^+ on that site in the current basis in order to grow the chain.
initial_block_left = Block(length=1, basis_size=model_d, operator_dict={
    "H": H1,
    "h1loc": Plq_right, #h1n1 h1n2
    "h2loc": Plq_right_deg, #h2n1 h2n2
})

initial_block_right = Block(length=1, basis_size=model_d, operator_dict={
    "H": H1,
    "h1loc": Plq_left,
    "h2loc": Plq_left_deg,
})


def enlarge_block(block, side="left"):
    """This function enlarges the provided Block by a single site, returning an
    EnlargedBlock.
    """
    mblock = block.basis_size
    o = block.operator_dict

    # Create the new operators for the enlarged block.  Our basis becomes a
    # Kronecker product of the Block basis and the single-site basis.  NOTE:
    # `kron` uses the tensor product convention making blocks of the second
    # array scaled by the first.  As such, we adopt this convention for
    # Kronecker products throughout the code.
    
    if side == "left":
        left_block = o["h1loc"] + o["h1loc"]
        right_block = Plq_right + Plq_right_deg
        
        enlarged_operator_dict = {
                "H": kron(o["H"], identity(model_d)) + kron(identity(mblock), H1) + H2(right_block, left_block),
                "h1loc": [kron(identity(mblock), op) for op in Plq_left],
                "h2loc": [kron(identity(mblock), op) for op in Plq_left_deg],
                }
    else:
        right_block = o["h1loc"] + o["h1loc"]
        left_block = Plq_left + Plq_left_deg
        
        enlarged_operator_dict = {
                "H": kron(o["H"], identity(model_d)) + kron(identity(mblock), H1) + H2(right_block, left_block),
                "h1loc": [kron(op, identity(mblock)) for op in Plq_right],
                "h2loc": [kron(op, identity(mblock)) for op in Plq_right_deg],

                }
        

    return EnlargedBlock(length=(block.length + 1),
                         basis_size=(block.basis_size * model_d),
                         operator_dict=enlarged_operator_dict)

def rotate_and_truncate(operator, transformation_matrix):
    """Transforms the operator to the new (possibly truncated) basis given by
    `transformation_matrix`.
    """
    return transformation_matrix.conjugate().transpose().dot(operator.dot(transformation_matrix))

def single_dmrg_step(sys, env, m):
    """Performs a single DMRG step using `sys` as the system and `env` as the
    environment, keeping a maximum of `m` states in the new basis.
    """
    assert is_valid_block(sys)
    assert is_valid_block(env)

    # Enlarge each block by a single site.
    sys_enl = enlarge_block(sys,'left')
    if sys is env:  # no need to recalculate a second time
        env_enl = sys_enl
    else:
        env_enl = enlarge_block(env,'right')

    assert is_valid_enlarged_block(sys_enl)
    assert is_valid_enlarged_block(env_enl)

    # Construct the full superblock Hamiltonian.
    m_sys_enl = sys_enl.basis_size
    m_env_enl = env_enl.basis_size
    sys_enl_op = sys_enl.operator_dict
    env_enl_op = env_enl.operator_dict
    superblock_hamiltonian = kron(sys_enl_op["H"], identity(m_env_enl)) + kron(identity(m_sys_enl), env_enl_op["H"]) + \
                             H2(sys_enl_op["h1loc"] + sys_enl_op["h2loc"], env_enl_op["h1loc"] + env_enl_op["h2loc"])

    # Call ARPACK to find the superblock ground state.  ("SA" means find the
    # "smallest in amplitude" eigenvalue.)
    (energy,), psi0 = eigsh(superblock_hamiltonian, k=1, which="SA")

    # Construct the reduced density matrix of the system by tracing out the
    # environment
    #
    # We want to make the (sys, env) indices correspond to (row, column) of a
    # matrix, respectively.  Since the environment (column) index updates most
    # quickly in our Kronecker product structure, psi0 is thus row-major ("C
    # style").
    psi0 = psi0.reshape([sys_enl.basis_size, -1], order="C")
    rho = np.dot(psi0, psi0.conjugate().transpose())

    # Diagonalize the reduced density matrix and sort the eigenvectors by
    # eigenvalue.
    evals, evecs = np.linalg.eigh(rho)
    possible_eigenstates = []
    for eval, evec in zip(evals, evecs.transpose()):
        possible_eigenstates.append((eval, evec))
    possible_eigenstates.sort(reverse=True, key=lambda x: x[0])  # largest eigenvalue first

    # Build the transformation matrix from the `m` overall most significant
    # eigenvectors.
    my_m = min(len(possible_eigenstates), m)
    transformation_matrix = np.zeros((sys_enl.basis_size, my_m), dtype='d', order='F')
    for i, (eval, evec) in enumerate(possible_eigenstates[:my_m]):
        transformation_matrix[:, i] = evec

    truncation_error = 1 - sum([x[0] for x in possible_eigenstates[:my_m]])
    print("truncation error:", truncation_error)

    # Rotate and truncate each operator.
    new_operator_dict = {}
    for name, op in sys_enl.operator_dict.items():
        if type(op) is list:
            new_operator_dict[name] = [rotate_and_truncate(o, transformation_matrix) for o in op]
        else:
            new_operator_dict[name] = rotate_and_truncate(op, transformation_matrix)

    new_left_block = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)
    
        # Rotate and truncate each operator.
    new_operator_dict = {}
    for name, op in sys_enl.operator_dict.items():
        if type(op) is list:
            new_operator_dict[name] = [rotate_and_truncate(o, transformation_matrix) for o in op]
        else:
            new_operator_dict[name] = rotate_and_truncate(op, transformation_matrix)

    new_right_block = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)

    return new_left_block, new_right_block, energy

def graphic(sys_block, env_block, sys_label="l"):
    """Returns a graphical representation of the DMRG step we are about to
    perform, using '=' to represent the system sites, '-' to represent the
    environment sites, and '**' to represent the two intermediate sites.
    """
    assert sys_label in ("l", "r")
    graphic = ("=" * sys_block.length) + "**" + ("-" * env_block.length)
    if sys_label == "r":
        # The system should be on the right and the environment should be on
        # the left, so reverse the graphic.
        graphic = graphic[::-1]
    return graphic

def infinite_system_algorithm(L, m):
    block = initial_block
    # Repeatedly enlarge the system by performing a single DMRG step, using a
    # reflection of the current block as the environment.
    while 2 * block.length < L:
        print("L =", block.length * 2 + 2)
        block, energy = single_dmrg_step(block, block, m=m)
        print("E/L =", energy / (block.length * 2))

def finite_system_algorithm(L, m_warmup, m_sweep_list):
    assert L % 2 == 0  # require that L is an even number

    # To keep things simple, this dictionary is not actually saved to disk, but
    # we use it to represent persistent storage.
    block_disk = {}  # "disk" storage for Block objects

    # Use the infinite system algorithm to build up to desired size.  Each time
    # we construct a block, we save it for future reference as both a left
    # ("l") and right ("r") block, as the infinite system algorithm assumes the
    # environment is a mirror image of the system.
    right_block = initial_block_right
    left_block = initial_block_left
    block_disk["l", right_block.length] = right_block
    block_disk["r", left_block.length] = left_block
    while left_block.length + right_block.length < L:
        # Perform a single DMRG step and save the new Block to "disk"
        print(graphic(left_block, right_block))
        left_block, right_block, energy = single_dmrg_step(left_block, right_block, m=m_warmup)
        print("E/L =", energy / (left_block.length * 2))
        block_disk["l", left_block.length] = left_block
        block_disk["r", right_block.length] = right_block

    # Now that the system is built up to its full size, we perform sweeps using
    # the finite system algorithm.  At first the left block will act as the
    # system, growing at the expense of the right block (the environment), but
    # once we come to the end of the chain these roles will be reversed.
    sys_label, env_label = "l", "r"
    sys_block = left_block; del left_block; del right_block  # rename the variable
    for m in m_sweep_list:
        while True:
            # Load the appropriate environment block from "disk"
            env_block = block_disk[env_label, L - sys_block.length - 2]
            if env_block.length == 1:
                # We've come to the end of the chain, so we reverse course.
                sys_block, env_block = env_block, sys_block
                sys_label, env_label = env_label, sys_label

            # Perform a single DMRG step.
            print(graphic(sys_block, env_block, sys_label))
            sys_block, env_block, energy = single_dmrg_step(sys_block, env_block, m=m)

            print("E/L =", energy / L)

            # Save the block from this step to disk.
            block_disk[sys_label, sys_block.length] = sys_block

            # Check whether we just completed a full sweep.
            if sys_label == "l" and 2 * sys_block.length == L:
                break  # escape from the "while True" loop

if __name__ == "__main__":
    np.set_printoptions(precision=10, suppress=True, threshold=10000, linewidth=300)

    #infinite_system_algorithm(L=100, m=20)
    finite_system_algorithm(L=10, m_warmup=2, m_sweep_list=[10, 20, 30, 40, 40])
# -*- coding: utf-8 -*-



=**-
truncation error: 0.8583081283480949
E/L = -0.5598880750969681
==**--
truncation error: 0.786641264673654
E/L = -0.33248941123175724
===**---
truncation error: 0.7695238551491654
E/L = -0.24058210133939537
====**----
truncation error: 0.6967872514594686
E/L = -0.19244877212415087
=====**---
truncation error: 0.2794379408547698
E/L = -0.19245168366497764
======**--
truncation error: 0.4729788700792512
E/L = -0.2159397569494851
-------**=
truncation error: 0.6008528691216841
E/L = -0.22808982537761832
------**==
truncation error: 0.5001180564594541
E/L = -0.2331657918000331
-----**===
truncation error: 0.2951497322475025
E/L = -0.22834786277414945
----**====
truncation error: 0.4081825734575245
E/L = -0.25023954697506723
---**=====
truncation error: 0.21017674882758342
E/L = -0.2674124712309606
--**======
truncation error: 0.3922568859830199
E/L = -0.2886944815697073
=**-------
truncation error: 0.5945336010552534
E/L = -0.29265000051378504
==**------
truncation error: 0.13065442682

array([0])

0

In [58]:
dezimal2binary(4)

array(['1', '0', '0'], dtype='<U1')